<a href="https://colab.research.google.com/github/ranmalmendis/AI_CHAT_BOT_OPEN_AI_ASSISTANTS_API/blob/main/Replicate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Started with Replicate
This notebook shows how to run models on [Replicate](https://replicate.com).

Last updated: 2024-01-17

>[Run a model from Python](#updateTitle=true&folderId=18DKj1jLZ00WQAwMRuIYM22n5yGpZzb1r&scrollTo=UXa9IwkeokWH)

>[Setup](#updateTitle=true&folderId=18DKj1jLZ00WQAwMRuIYM22n5yGpZzb1r&scrollTo=ep0A2pLDnoWK)

>[Run a model](#updateTitle=true&folderId=18DKj1jLZ00WQAwMRuIYM22n5yGpZzb1r&scrollTo=Ax6xbVZOpnaV)

>[Run a model in the background](#updateTitle=true&folderId=18DKj1jLZ00WQAwMRuIYM22n5yGpZzb1r&scrollTo=xLvskaxwtswn)

>[Run a model in the background and get a webhook](#updateTitle=true&folderId=18DKj1jLZ00WQAwMRuIYM22n5yGpZzb1r&scrollTo=X7ZZHjNrunwr)

>[Compose models into a pipeline](#updateTitle=true&folderId=18DKj1jLZ00WQAwMRuIYM22n5yGpZzb1r&scrollTo=xyfeLTILu3ad)

>[Get streaming output from a running model](#updateTitle=true&folderId=18DKj1jLZ00WQAwMRuIYM22n5yGpZzb1r&scrollTo=WM47DByLrk5l)

>[Cancel a prediction](#updateTitle=true&folderId=18DKj1jLZ00WQAwMRuIYM22n5yGpZzb1r&scrollTo=K7kZwHzLwWoM)

>[Load output files](#updateTitle=true&folderId=18DKj1jLZ00WQAwMRuIYM22n5yGpZzb1r&scrollTo=5cuMDPg1xjQZ)

>[Next steps](#updateTitle=true&folderId=18DKj1jLZ00WQAwMRuIYM22n5yGpZzb1r&scrollTo=vz6FASGXsefP)



# Setup

To run this notebook, you’ll need to create a [Replicate](https://replicate.com) account and install the Replicate python client.

In [1]:
# install replicate client
!pip install replicate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00


# Authenticate
Before running any Python scripts that use the API, you need to set your Replicate API token in your environment.

Grab your token from replicate.com/account and set it as an environment variable:

export REPLICATE_API_TOKEN=<your token>
We recommend not adding the token directly to your source code, because you don't want to put your credentials in source control. If anyone used your API key, their usage would be charged to your account.

In [ ]:
# get a token: https://replicate.com/account
from getpass import getpass
import os

REPLICATE_API_TOKEN = getpass()
os.environ["REPLICATE_API_TOKEN"] = r8_X945edtx9dM3JMwZGsVR9syBFuxjbbz4ZUkXS

# Run a model
You can run any public model on Replicate from your Python code. The following example runs [stability-ai/stable-diffusion](https://replicate.com/stability-ai/stable-diffusion):

In [ ]:
import replicate

output = replicate.run(
  "stability-ai/sdxl:39ed52f2a78e934b3ba6e2a89f5b1c712de7dfea535525255b1aa35c5565e08b",
  input={"prompt": "an iguana on the beach, pointillism"}
)

output

In [ ]:
from IPython.display import Image
Image(url=output[0])

Some models receive images as inputs. To pass a file as an input, use a file handle or URL:

In [ ]:
# image = open("mystery.jpg", "rb")
# or...
# image = "https://example.com/mystery.jpg"
#
# for this example, let's use the URL from the previous prediction:
image = output[0]

replicate.run(
  "andreasjansson/blip-2:4b32258c42e9efd4288bb9910bc532a69727f9acd26aa08e175713a0a857a608",
  input={"image": output[0], "prompt": "what's in this picture?"}
)

# Run a model in the background
You can start a model and run it in the background:

In [ ]:
model = replicate.models.get("meta/musicgen")
version = model.versions.get("b05b1dff1d8c6dc63d14b0cdb42135378dcb87f6373b0d3d341ede46e59e2b38")

prediction = replicate.predictions.create(
    version=version,
    input={"prompt":"Edo25 major g melodies that sound triumphant and cinematic. Leading up to a crescendo that resolves in a 9th harmonic"})

In [ ]:
prediction

In [ ]:
prediction.status

In [ ]:
dict(prediction).keys()

In [ ]:
prediction.wait()
prediction.reload()

In [ ]:
prediction.status

In [ ]:
import IPython
IPython.display.Audio(prediction.output)

# Run a model in the background and get a webhook

You can run a model and get a webhook when it completes, instead of waiting for it to finish.

If you're working locally, we'd recommend using [ngrok](https://ngrok.com/download) for debugging webhooks. It allows you to tunnel your localhost to a public domain.

In [ ]:
model = replicate.models.get("jd7h/photomaker")
version = model.versions.get("79b21a43649e0d08a7cbf9b553420047456de4492efb1a7c73d500c1c8812fc6")
prediction = replicate.predictions.create(
    version=version,
    input={
        "seed": 1143488585,
        "prompt": "A photo of a scientist winning the Nobel prize img",
        "num_steps": 50,
        "style_name": "Photographic (Default)",
        "input_image": "https://replicate.delivery/pbxt/KF73Yow3WJw7ebPRq1tPohZXN0bjoAjbNcXiJzeineGGAv1A/newton_0.jpg",
        "num_outputs": 1,
        "guidance_scale": 5,
        "negative_prompt": "nsfw, lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry",
        "style_strength_ratio": 20
    },
    webhook="https://example.com/your-webhook",
    webhook_events_filter=["completed"]
)

# Compose models into a pipeline
You can run a model and feed the output into another model:

In [ ]:
sdxl = replicate.models.get("stability-ai/sdxl").versions.get("39ed52f2a78e934b3ba6e2a89f5b1c712de7dfea535525255b1aa35c5565e08b")
esrgan = replicate.models.get("nightmareai/real-esrgan").versions.get("42fed1c4974146d4d2414e2be2c5277c7fcf05fcc3a73abf41610695738c1d7b")
image = replicate.predictions.create(version=sdxl, input={"prompt": "avocado armchair"})
image.wait()
upscaled_image = replicate.predictions.create(version=esrgan, input={"image": image.output[0]})
upscaled_image.wait()

In [ ]:
from IPython.display import Image
Image(url=upscaled_image.output)

# Get streaming output from a running model
Some models stream output as the model is running. They will return an iterator, and you can iterate over that output:

In [ ]:
iterator = replicate.run(
  "mistralai/mixtral-8x7b-instruct-v0.1",
  input={"prompt": "Who was Dolly the sheep?"},
)
for text in iterator:
      print(text)

# Cancel a prediction
You can cancel a running prediction:

In [ ]:
model = replicate.models.get("stability-ai/stable-video-diffusion")
version = model.versions.get("3f0457e4619daac51203dedb472816fd4af51f3149fa7a9e0b5ffcf1b8172438")

prediction = replicate.predictions.create(
    version=version,
    input={
      "cond_aug": 0.02,
      "decoding_t": 7,
      "input_image": "https://replicate.delivery/pbxt/JvLi9smWKKDfQpylBYosqQRfPKZPntuAziesp0VuPjidq61n/rocket.png",
      "video_length": "14_frames_with_svd",
      "sizing_strategy": "maintain_aspect_ratio",
      "motion_bucket_id": 127,
      "frames_per_second": 6
    })

In [ ]:
prediction.status

In [ ]:
prediction.cancel()

In [ ]:
prediction.reload()

In [ ]:
prediction.status

In [ ]:
predictions = replicate.predictions.list()

[p.status for p in predictions[:10]]

# Load output files
Output files are returned as HTTPS URLs. You can load an output file as a buffer:

In [ ]:
from urllib.request import urlretrieve

model = replicate.models.get("stability-ai/stable-video-diffusion")
version = model.versions.get("3f0457e4619daac51203dedb472816fd4af51f3149fa7a9e0b5ffcf1b8172438")

prediction = replicate.predictions.create(
    version=version,
    input={
      "cond_aug": 0.02,
      "decoding_t": 7,
      "input_image": "https://replicate.delivery/pbxt/JvLi9smWKKDfQpylBYosqQRfPKZPntuAziesp0VuPjidq61n/rocket.png",
      "video_length": "14_frames_with_svd",
      "sizing_strategy": "maintain_aspect_ratio",
      "motion_bucket_id": 127,
      "frames_per_second": 6
    })
prediction.wait()

urlretrieve(prediction.output, "/tmp/out.mp4")

In [ ]:
from IPython.display import Video

Video(prediction.output)

# Next steps

- Read the [full Python client documentation on GitHub](https://github.com/replicate/replicate-python#readme).

- Explore our collection of hosted [models](https://replicate.com/explore)

- Note that you can also run models with the raw HTTP API. Refer to the [HTTP API reference](https://replicate.com/docs/reference/http) for more details.